### Importing Libraries

In [ ]:
import os
import cv2
import numpy as np
import pydot
import graphviz
from collections import deque
import matplotlib.pyplot as plt
import pickle
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.layers import AveragePooling2D, Flatten, Dense, Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers.legacy import SGD
import tensorflow
from sklearn.metrics import classification_report
from imutils import paths

In [ ]:
# Set the path to your dataset folder in Google Drive
dataset_path = '/content/drive/MyDrive/Video_Project/dataset'

In [ ]:
# Initialize the labels and data lists
LABELS = []
data = []

### Reading and Processing images from subfolders in a specified directory and stores the processed images and their class labels in lists

In [ ]:
# Loop over the subfolders
for class_folder in os.listdir(dataset_path):
    print(class_folder)
    class_folder_path = os.path.join(dataset_path, class_folder)
    if os.path.isdir(class_folder_path):
        # Loop over the images in the subfolder
        for image_file in os.listdir(class_folder_path):
            if image_file.endswith((".gif", ".aspx", ".ipynb_checkpoints")):  # Skip images with ".gif", ".aspx", ".ipynb_checkpoints" extension
                continue
            image_path = os.path.join(class_folder_path, image_file)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (224, 224))
            data.append(image)
            LABELS.append(class_folder)



shooting
wrestling
table_tennis
volleyball
swimming
wwe
motogp
weight_lifting
tennis
fencing
kabaddi
gymnastics
boxing
hockey
football
ice_hockey
cricket
formula1
chess
baseball
badminton
basketball


In [ ]:
len(data)

14512

In [ ]:
len(LABELS)

14512

In [ ]:
import pickle
with open('/content/drive/MyDrive/Video_Project/data.pickle', 'wb') as f:
    pickle.dump(data, f)

with open('/content/drive/MyDrive/Video_Project/labels.pickle', 'wb') as f:
    pickle.dump(LABELS, f)

In [ ]:
import pickle
# Load data from the pickle file
with open('/content/drive/MyDrive/Video_Project/data.pickle', 'rb') as f:
    data = pickle.load(f)

# Load LABELS from the pickle file
with open('/content/drive/MyDrive/Video_Project/labels.pickle', 'rb') as f:
    LABELS = pickle.load(f)

### Data Preprocessing

In [ ]:
# Convert data and labels to NumPy arrays
data = np.array(data)
labels = np.array(LABELS)

In [ ]:
# Perform label binarization
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

In [ ]:
import pickle
with open('labelBinarizer.pickle', 'wb') as f:
    pickle.dump(lb, f)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Video_Project/data_np.pickle', 'wb') as f:
    pickle.dump(data, f)

with open('/content/drive/MyDrive/Video_Project/labels_np.pickle', 'wb') as f:
    pickle.dump(labels, f)

In [ ]:
import pickle
# Load data from the pickle file
with open('/content/drive/MyDrive/Video_Project/data_np.pickle', 'rb') as f:
    data = pickle.load(f)

# Load labels from the pickle file
with open('/content/drive/MyDrive/Video_Project/labels_np.pickle', 'rb') as f:
    labels = pickle.load(f)

### Splitting the Data into tain and test set

In [ ]:
# Partition the data into training and testing sets
(X_train, X_test, y_train, y_test) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)

In [ ]:
print(data)
print(data.shape)

In [ ]:
print(labels)
print(labels.shape)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 1 ... 0 0 0]]
(14512, 22)


### Data Augmentation

In [ ]:
# Initialize the training data augmentation object
trainAug = ImageDataGenerator(
    rotation_range=30,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    horizontal_flip=True,
    fill_mode="nearest")

# Initialize the validation/testing data augmentation object
valAug = ImageDataGenerator()

# Define the ImageNet mean subtraction (in RGB order) and set the mean subtraction value for each of the data augmentation objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

### Constructing a neural network model using the ResNet-50 architecture and setting the layers in the 'conv5_' group to be trainable for fine tuning

In [ ]:
# Load the ResNet-50 network, excluding the head FC layers
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

# Construct the head of the model to be placed on top of the base model
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(2048, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(1024, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(22, activation="softmax")(headModel)

# Place the head FC model on top of the base model
model = Model(inputs=baseModel.input, outputs=headModel)

for basemodelLayers in baseModel.layers:
    if basemodelLayers.name.startswith('conv5_'):
        basemodelLayers.trainable = True
    else:
        basemodelLayers.trainable = False

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
print(model.summary())

### Training the model

In [ ]:
# Compile the model
# opt = SGD(learning_rate=1e-4, momentum=0.9, decay=1e-4 / 50)
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the head of the network for a few epochs
print("[INFO] Training the head...")
H = model.fit(
    x=trainAug.flow(X_train, y_train, batch_size=32),
    steps_per_epoch=len(X_train) // 32,
    validation_data=valAug.flow(X_test, y_test),
    validation_steps=len(X_test) // 32,
    epochs=50)

model.save("model_after_50_epochs")

[INFO] Training the head...


In [ ]:
model = tensorflow.keras.models.load_model("model_after_50_epochs")
with open('labelBinarizer.pickle', 'rb') as f:
    lb = pickle.load(f)
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
Queue = deque(maxlen = 128)

### Processing a video file, makes real-time predictions using a deep learning model, and annotates the video frames with activity labels

In [ ]:
capture_video = cv2.VideoCapture("boxing.mp4")
writer = None
(Width, Height) = (None, None)
while True:
    (taken, frame) = capture_video.read()
    if not taken:
        break
    if Width is None or Height is None:
        (Width, Height) = frame.shape[:2]

    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Queue.append(preds)
    results = np.array(Queue).mean(axis = 0)
    i = np.argmax(results)
    label = lb.classes_[i]
    text = "They are playing : {}".format(label)
    cv2.putText(output,text, (45, 60), cv2.FONT_HERSHEY_SIMPLEX,1.25, (255,0,0), 5)

    if writer is None:
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter("outputvideo", fourcc, 30, (Width, Height), True)
    writer.write(output)
    cv2.imshow("In progress", output)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

print("Finalizing....")
writer.release()
capture_video.release()